## ChemicalFun
Generate reactions based on species and symbols provided in a excel spreadsheet. Then calculate the reaction properties based on thermodynamic data from a database using ThermoFun.

In [ ]:
import chemicalfun as cf
import pandas as pd
import thermohubclient as thc

In [ ]:
# Read the Excel file
df = pd.read_excel('substances_formulas_symbols.xlsx', sheet_name='CASHplus', engine='openpyxl')

# Concatenate the data from columns 'master_formulas' and 'dependent_formulas' into a python list 'formulas'
formulas = df['master_formulas'].tolist() + df['dependent_formulas'].tolist()

# Concatenate the data from columns 'master_symbols' and 'dependent_symbols' into a python list 'symbols'
symbols = df['master_symbols'].tolist() + df['dependent_symbols'].tolist()

# Remove nan values from the lists
formulas = [formula for formula in formulas if pd.notna(formula)]
symbols = [symbol for symbol in symbols if pd.notna(symbol)]

print("Formulas:", formulas)
print("Symbols:", symbols)

In [ ]:
chemicalReactions = cf.ChemicalReactions(formulas, symbols)

reactions = chemicalReactions.generateReactions() # returns the reactions list as a list of tuples ('substance', coefficient)

# can be transformed to a list of dictionaries, with reaction substances as keys and the reaction coefficients as values
reactions_dic = [{el[0]: el[1] for el in r} for r in reactions] 

print(chemicalReactions.printReactions())
print(reactions_dic)

In [ ]:
import thermofun as tf
cemdata18 = tf.Database('CEMDATA18-subset-thermofun.json')

In [ ]:
engine = tf.ThermoEngine(cemdata18)

In [ ]:
for r in chemicalReactions.stringReactions():
    result = engine.thermoPropertiesReaction(298.15, 1, r).log_equilibrium_constant.val
    print(f"Reaction: {r}, logK: {result}")
    print(result)